<a href="https://colab.research.google.com/github/the-SQuAD-squad/IR-QA/blob/main/QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# dataset is copyed to public git repo for fast access within colab 
! git clone https://github.com/buoi/question-answered.git
! unzip question-answered/training_set.json.zip

Cloning into 'question-answered'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
Archive:  question-answered/training_set.json.zip
  inflating: training_set.json       
  inflating: __MACOSX/._training_set.json  


In [ ]:
import json

with open("training_set.json", "r") as f:
    json_file = json.load(f)

data = json_file["data"]
for i in range(10):
  print(data[i]["title"])

data[8]["paragraphs"][0]

In [ ]:
# df creation
import pandas as pd

passages = []
ids = []
for document in data:
  for par in document['paragraphs']:
    passages.append(par['context'])
    for qas in par['qas']:

      dataframe_row = {
          'id' : qas['id'],
          ' '
          }

dict{dcomunetn_title: }
#due pd dataframes: 
# document_title, passage

# id, passage, question, answer
#   (doc: title/1,2,3 , pass_1,2,3)